## Preparation et nettoyage 🧼

Telechargement du dataset

In [0]:
from pyspark.sql import functions as F

In [0]:
filepath = "s3://full-stack-bigdata-datasets/Big_Data/Project_Steam/steam_game_output.json"
df = spark.read.format('json').load(filepath, multiline=True)
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

Selection des infos utiles:

In [0]:
df= df.select("*", "data.*").drop("data")
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- appid: long (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ccu: long (nullable = true)
 |-- developer: string (nullable = true)
 |-- discount: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- header_image: string (nullable = true)
 |-- initialprice: string (nullable = true)
 |-- languages: string (nullable = true)
 |-- name: string (nullable = true)
 |-- negative: long (nullable = true)
 |-- owners: string (nullable = true)
 |-- platforms: struct (nullable = true)
 |    |-- linux: boolean (nullable = true)
 |    |-- mac: boolean (nullable = true)
 |    |-- windows: boolean (nullable = true)
 |-- positive: long (nullable = true)
 |-- price: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- required_age: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- tags: struct (nul

Changement du format de la date sous format "yyyy/mm/dd" et /100 du prix

In [0]:
df = df.withColumn('release_date_clean', F.to_date(F.col("release_date"), format="yyyy/MM/dd"))
df = df.withColumn("year", F.year(df['release_date_clean']))
df = df.drop("release_date")
df = df.withColumns({'initialprice':F.col('initialprice')/100,"price":F.col("price")/100})

## 📊Analyse Macro

* Quel éditeur a publié le plus de jeux sur Steam et son prix moyen  ? (TOP 10)

In [0]:
publisher_count = df.withColumn("publisher", F.regexp_replace(F.col("publisher"), ", Inc", " Inc"))\
                    .withColumn("publisher", F.regexp_replace(F.col("publisher"), ", INC", " Inc"))\
                    .withColumn("publisher", F.regexp_replace(F.col("publisher"), ", Ltd", " ltd"))\
                    .withColumn("publisher", F.regexp_replace(F.col("publisher"), ", LTD", " LTD"))\
                    .withColumn("publisher", F.regexp_replace(F.col("publisher"), ", LLC", " LLC"))\
                    .withColumn('publisher', F.explode(F.split("publisher",", ")))\
                    .groupBy('publisher')\
                    .agg(F.count('*').alias('gameCount'),
                         F.mean('price').alias('price_mean'))\
                    .orderBy(F.desc('gameCount'))

publisher_count.limit(9).display()

publisher,gameCount,price_mean
Big Fish Games,422,9.497582938388526
8floor,202,4.994950495049513
SEGA,187,16.39850267379673
Square Enix,151,20.52774834437081
Strategy First,151,6.9243708609271595
THQ Nordic,144,16.184652777777767
Choice of Games,140,5.346428571428578
Sekai Project,136,14.18176470588236
Plug In Digital,136,8.196764705882359


Databricks visualization. Run in Databricks to view.

Big fish a produit 422 jeux avec un prix moyen de 9.49$, suivit par 8floor avec 202 jeux et SEGA avec 187 jeux.

* Quels sont les jeux les mieux notés ?

In [0]:
display(df.select('name','positive').sort(F.col("positive").desc()).limit(10))

name,positive
Counter-Strike: Global Offensive,5943345
Dota 2,1534895
Grand Theft Auto V,1229265
PUBG: BATTLEGROUNDS,1185361
Terraria,1014711
Tom Clancy's Rainbow Six Siege,942910
Garry's Mod,861240
Team Fortress 2,846407
Rust,732513
Left 4 Dead 2,643836


Databricks visualization. Run in Databricks to view.

Counter Strike, Dota, GTA 5

* Y a-t-il des années avec plus de sorties ? Y a-t-il eu plus ou moins de sorties de jeux pendant le Covid par exemple ?

In [0]:
releases_y = df.groupby('year').count().sort(F.col('count').desc())
display(releases_y)

year,count
2021,8805
2020,8287
2018,7663
2022,7451
2019,6949
2017,6006
2016,4176
2015,2566
2014,1550
2013,469


Databricks visualization. Run in Databricks to view.

Depuis 2013, on voit qu'il y'a une tendance a la hausse en terme de publication de jeux sur la platforme steam.
Un premier pic est connu en 2018,puis un autre en 2020-2021.

* Comment sont distribués les prix ? Y a-t-il beaucoup de jeux avec une réduction ?

In [0]:
distrib_prix = df.groupBy('price').count().sort(F.col('count').desc())
display(distrib_prix)
df_price = df.select(['initialprice', 'discount', 'price'])
total_count = df_price.count()
nb_discount = df_price.filter(df_price['discount'] > 0).count()
proportion = (nb_discount / total_count) * 100
discount_df = spark.createDataFrame([("With Discount", nb_discount), ("Without Discount", total_count - nb_discount)], ["Category", "Count"])
display(discount_df)

price,count
0.0,7780
4.99,6250
9.99,6126
0.99,5243
1.99,4193
2.99,3639
14.99,2878
19.99,2820
3.99,2530
6.99,1856


Databricks visualization. Run in Databricks to view.

Category,Count
With Discount,2518
Without Discount,53173


Databricks visualization. Run in Databricks to view.

Un peu plus de 4% des jeux sont en promo (2518)

*   Quelles sont les langues les plus représentées ?

In [0]:
df = df.withColumn('languages', F.split(F.col('languages'), ','))
languages=df.select(F.explode('languages').alias('Language'))
display(languages.groupBy('Language').count().sort(F.col('count').desc()).limit(10))

Language,count
English,54646
German,13996
French,13406
Russian,12839
Spanish - Spain,12224
Simplified Chinese,12213
Japanese,10170
Italian,9297
Portuguese - Brazil,6739
Korean,6575


Databricks visualization. Run in Databricks to view.

Anglais, Allemand, Francais, c'est plus de 50% des langues representés.

## Analyse des genres

* Quels sont les genres les plus représentés ?

In [0]:
genre = df.withColumn('genre_list', F.split(F.trim(df['genre']), ', '))
genre=genre.withColumn('Genre', F.explode(genre['genre_list']))
display(genre.groupBy('Genre').count().sort(F.col('count').desc()).limit(10))

Genre,count
Indie,39681
Action,23759
Casual,22086
Adventure,21431
Strategy,10895
Simulation,10836
RPG,9534
Early Access,6145
Free to Play,3393
Sports,2666


Databricks visualization. Run in Databricks to view.

Top 3 genres : Indie, action, casual

* Existe-t-il des genres qui ont un meilleur ratio d'avis positifs/négatifs ?

In [0]:
df_exploded_genres = df.select(F.explode(F.split('genre', ',')).alias('genre'), 'positive', 'negative')
df_exploded_genres = df_exploded_genres.withColumn('ratio', F.round(F.col('positive') / F.col('negative')))
df_filtered = df_exploded_genres.filter(F.col('ratio') > 1)
display(df_filtered)

genre,positive,negative,ratio
Action,201215,5199,39.0
Action,27,5,5.0
Adventure,27,5,5.0
Indie,27,5,5.0
Adventure,4032,646,6.0
Indie,4032,646,6.0
RPG,4032,646,6.0
Strategy,4032,646,6.0
Action,1575,115,14.0
Indie,1575,115,14.0


Databricks visualization. Run in Databricks to view.

Le genre ayant le meilleur ratio de notes positifs sur les notes négatifs est le genre Action, puis Indie, puis Adventure.

* Certains éditeurs ont-ils des genres favoris ?

In [0]:
# voyons deja les genres par editeurs :
Genre_edit = genre\
    .groupBy('publisher','Genre')\
        .count().sort(F.col('count').desc())
display(Genre_edit)

publisher,Genre,count
Big Fish Games,Casual,418
Big Fish Games,Adventure,392
8floor,Casual,202
Choice of Games,RPG,139
Choice of Games,Indie,136
HH-Games,Casual,132
Laush Studio,Indie,124
Choice of Games,Adventure,112
,Indie,106
Alawar Entertainment,Casual,105


Big Fish Games les deux genres les plus prédominants sont "casual" et "adventure"

* Quels sont les genres les plus lucratifs ?

In [0]:
p_genre = genre\
    .groupBy('Genre')\
        .sum('price')\
            .sort(F.col('sum(price)').desc()).limit(10)
display(p_genre)

Genre,sum(price)
Indie,260630.3599998896
Action,183587.68999997684
Adventure,171581.78999998886
Casual,123835.83000003877
Simulation,98516.54000001294
Strategy,91572.01000000977
RPG,86212.95000000563
Early Access,53757.94999999832
Sports,23855.290000000914
Racing,17716.449999999906


Databricks visualization. Run in Databricks to view.

Le genre le plus lucratife est le style Indie, suivie de celui d'action et Aventure

## Analyse de la plateforme

* La plupart des jeux sont-ils disponibles sur Windows/Mac/Linux ?

In [0]:
# Getting nested field platform value and affecting 1 if True and 0 if False
platforms = df\
    .withColumn("Linux", F.when(F.col('platforms').getField('linux'), 1).otherwise(0))\
    .withColumn("Mac", F.when(F.col('platforms').getField('mac'), 1).otherwise(0))\
    .withColumn("Windows", F.when(F.col('platforms').getField('windows'), 1).otherwise(0))

# Calculating the sum of games on each platform
sum_platforms = platforms\
    .agg(F.sum('Mac').alias('Total Mac'), F.sum('Linux').alias('Total Linux'), F.sum('Windows').alias('Total Windows'))

# Number of rows in df
total_games = df.count()

# Displaying the results with total number of games
display(sum_platforms\
    .withColumn('Total number of games', F.lit(total_games)) # Adding the total number of games to the results
    .withColumn('Percentage Mac', F.round(F.col('Total Mac') / total_games * 100,2))
    .withColumn('Percentage Linux', F.round(F.col('Total Linux') / total_games * 100,2))
    .withColumn('Percentage Windows', F.round(F.col('Total Windows') / total_games * 100,2)))

Total Mac,Total Linux,Total Windows,Total number of games,Percentage Mac,Percentage Linux,Percentage Windows
12770,8458,55676,55691,22.93,15.19,99.97


Databricks visualization. Run in Databricks to view.

99.9% des jeux sont sur Windows